In [ ]:
import boto3
import datetime
from typing import Dict, List, Any, Optional
from decimal import Decimal
import uuid
from datetime import timedelta
import os 
from boto3.dynamodb.conditions import Attr,Key
dynamodb = boto3.resource(
    'dynamodb',
    endpoint_url=os.getenv("DYNAMO_URL"),  
    region_name=os.getenv("REGION_NAME")
)

# List tables
for table in dynamodb.tables.all():
    print(table.name)


VNA_history


In [ ]:
TABLE_NAME = os.getenv("TABLE_NAME")
table = None

In [ ]:
table = boto3.resource(
    'dynamodb',
    endpoint_url=os.getenv("DYNAMO_URL"),
    region_name=os.getenv("REGION_NAME")
).Table(os.getenv("TABLE_NAME"))

In [ ]:
# def create_table(table_name=TABLE_NAME):
#     """Create the DynamoDB table with appropriate keys and indexes."""
#     global table
    
#     if table_name in [table.name for table in dynamodb.tables.all()]:
#         print(f"Table {table_name} already exists")
#         table = dynamodb.Table(table_name)
#         return table
        
#     print(f"Creating table {table_name}")
#     table = dynamodb.create_table(
#         TableName=table_name,
#         KeySchema=[
#             {
#                 'AttributeName': 'id',
#                 'KeyType': 'HASH'  
#             }
#         ],
#         AttributeDefinitions=[
#             {
#                 'AttributeName': 'id',
#                 'AttributeType': 'S'
#             },
#             {
#                 'AttributeName': 'conversation_id',
#                 'AttributeType': 'S'
#             },
#             {
#                 'AttributeName': 'created_at',
#                 'AttributeType': 'S'
#             }
#         ],
#         GlobalSecondaryIndexes=[
#             {
#                 'IndexName': 'conversation_index',
#                 'KeySchema': [
#                     {
#                         'AttributeName': 'conversation_id',
#                         'KeyType': 'HASH'
#                     },
#                     {
#                         'AttributeName': 'created_at',
#                         'KeyType': 'RANGE'
#                     }
#                 ],
#                 'Projection': {
#                     'ProjectionType': 'ALL'
#                 },
#                 'ProvisionedThroughput': {
#                     'ReadCapacityUnits': 5,
#                     'WriteCapacityUnits': 5
#                 }
#             }
#         ],
#         ProvisionedThroughput={
#             'ReadCapacityUnits': 5,
#             'WriteCapacityUnits': 5
#         }
#     )
    
#     table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
#     print(f"Table {table_name} created successfully")
    
#     return table

In [ ]:
def check_table_exists(dynamodb, table_name):
    """Check if the table exists in DynamoDB."""
    try:
        existing_tables = [t.name for t in dynamodb.tables.all()]
        exists = table_name in existing_tables
        print(f"Table '{table_name}' exists: {exists}")
        return exists
    except Exception as e:
        print(f"Error checking table existence: {str(e)}")
        return False

In [ ]:
def save_chat_history(
    conversation_id: str,
    user_input: str,
    prompt_token: Optional[int] = None,
    completion_token: Optional[int] = None,
    total_token: Optional[int] = None,
    start_time: Optional[datetime.datetime] = None,
    end_time: Optional[datetime.datetime] = None,
    execution_time: Optional[Decimal] = None,
    response: Optional[str] = None,
    category_id: Optional[str] = None,
    category_name: Optional[str] = None,
    source: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    if not check_table_exists(dynamodb, TABLE_NAME):
        raise Exception(f"DynamoDB table '{TABLE_NAME}' does not exist.")
    now = datetime.datetime.now()
    start_time = start_time or now
    end_time = end_time or now
    execution_time = execution_time or Decimal((end_time - start_time).total_seconds())

    category_code = {
        "category_id": category_id or "general",
        "category_name": category_name or "general"
    }

    question_id = f"user-{conversation_id}-{int(start_time.timestamp())}"
    user_id = str(uuid.uuid4())
    ai_id = str(uuid.uuid4())

    # Save user message
    user_item = {
        "id": user_id,
        "conversation_id": conversation_id,
        "question_id": question_id,
        "type": "HUMAN-MESSAGE",
        "user_content": user_input,
        "created_at": start_time.isoformat(),
        "updated_at": end_time.isoformat(),
        "prompt_token": prompt_token,
        "completion_token": completion_token,
        "total_token": total_token,
        "responded_in_s": execution_time,
        "category_code": category_code
    }
    table.put_item(Item=user_item)

    if response:
        ai_item = {
            "id": ai_id,
            "conversation_id": conversation_id,
            "question_id": question_id,
            "type": "AI-MESSAGE",
            "ai_content": response,
            "created_at": start_time.isoformat(),
            "updated_at": end_time.isoformat(),
            "prompt_token": prompt_token,
            "completion_token": completion_token,
            "total_token": total_token,
            "responded_in_s": execution_time,
            "category_code": category_code
        }
        if source:
            ai_item["source"] = source

        table.put_item(Item=ai_item)

    return {
        "conversation_id": conversation_id,
        "question_id": question_id
    }

In [ ]:
def get_conversation_history(conversation_id: str, limit: int = 50) -> List[Dict[str, Any]]:
    """
    Retrieve and format conversation history where each message is a separate row.

    Args:
        conversation_id: ID of the conversation to retrieve.
        limit: Max number of messages to return.

    Returns:
        List of formatted message dictionaries.
    """


    response = table.query(
        IndexName='conversation_index',
        KeyConditionExpression=Key('conversation_id').eq(conversation_id),
        ScanIndexForward=True,  
        Limit=limit
    )

    items = [item for item in response.get('Items', []) if item.get("type") in ["HUMAN-MESSAGE", "AI-MESSAGE"]]

    # Group by question_id and sort: HUMAN first, then AI
    grouped = {}
    for item in items:
        qid = item['question_id']
        grouped.setdefault(qid, []).append(item)

    formatted = []
    for qid, msgs in grouped.items():
        msgs_sorted = sorted(msgs, key=lambda x: 0 if x['type'] == "HUMAN-MESSAGE" else 1)
        for msg in msgs_sorted:
            content = msg.get("user_content") if msg['type'] == "HUMAN-MESSAGE" else msg.get("ai_content")
            category_code = msg.get("category_code", {})
            formatted.append({
                "id": msg.get("id"),
                "ConversationID": msg.get("conversation_id"),
                "type": msg.get("type"),
                "QuestionID": msg.get("question_id"),
                "Content": content,
                "Created_at": msg.get("created_at"),
                "Cpdated_at": msg.get("updated_at"),
                "CategoryID": category_code.get("category_id", "general"),
                "CategoryName": category_code.get("category_name", "general"),
                "tokens": {
                    "prompt": msg.get("prompt_token"),
                    "completion": msg.get("completion_token"),
                    "total": msg.get("total_token"),
                },
                "ResponseTime": msg.get("responded_in_s"),
                "Source": msg.get("source", None)
            })
    print(f"Retrieved {len(formatted)} messages for conversation {conversation_id}")
    return formatted

In [ ]:
def delete_conversation(conversation_id: str) -> int:
    """
    Delete all messages in a conversation
    Args:
        conversation_id: ID of the conversation to delete
    Returns:
        Number of messages deleted
    """


    deleted_count = 0
    last_evaluated_key = None

    while True:
        query_kwargs = {
            "IndexName": "conversation_index",
            "KeyConditionExpression": boto3.dynamodb.conditions.Key('conversation_id').eq(conversation_id),
        }
        if last_evaluated_key:
            query_kwargs["ExclusiveStartKey"] = last_evaluated_key

        response = table.query(**query_kwargs)
        items = response.get('Items', [])

        for item in items:
            table.delete_item(Key={"id": item['id']})
            deleted_count += 1
            print(f"Deleted item with ID: {item['id']}")

        if 'LastEvaluatedKey' in response:
            last_evaluated_key = response['LastEvaluatedKey']
        else:
            break

    print(f"Deleted a total of {deleted_count} messages from conversation {conversation_id}")
    return deleted_count


In [ ]:
def list_conversation(
    conversation_id: str,
    limit: Optional[int] = None,
    days: int = 7
) -> List[Dict[str, Any]]:
    """
    Retrieve recent conversation messages (one per row), sorted newest first.

    Args:
        conversation_id: The ID of the conversation to retrieve history for.
        limit: Optional number of most recent items to return.
        days: Number of days to look back (default: 7 days), used if limit is None.

    Returns:
        List of individual message items sorted by created_at descending.
    """


    # Determine query parameters
    if limit is None:
        # Calculate cutoff date based on the 'days' parameter
        cutoff_date = (datetime.datetime.now() - datetime.timedelta(days=days)).isoformat()
        response = table.query(
            IndexName='conversation_index',
            KeyConditionExpression=Key('conversation_id').eq(conversation_id) &
                                Key('created_at').gte(cutoff_date),
            ScanIndexForward=False  
        )
    else:
        response = table.query(
            IndexName='conversation_index',
            KeyConditionExpression=Key('conversation_id').eq(conversation_id),
            ScanIndexForward=False,  
            Limit=limit
        )

    items = response.get('Items', [])
    formatted = []
    for msg in items:
        content = msg.get("user_content") if msg.get("type") == "HUMAN-MESSAGE" else msg.get("ai_content")
        category_code = msg.get("category_code", {})
        formatted.append({
                "id": msg.get("id"),
                "ConversationID": msg.get("conversation_id"),
                "type": msg.get("type"),
                "QuestionID": msg.get("question_id"),
                "Content": content,
                "Created_at": msg.get("created_at"),
                "Cpdated_at": msg.get("updated_at"),
                "CategoryID": category_code.get("category_id", "general"),
                "CategoryName": category_code.get("category_name", "general"),
                "tokens": {
                    "prompt": msg.get("prompt_token"),
                    "completion": msg.get("completion_token"),
                    "total": msg.get("total_token"),
                },
                "ResponseTime": msg.get("responded_in_s"),
                "Source": msg.get("source", None)
        })

    print(f"Retrieved {len(formatted)} messages for conversation {conversation_id}")
    return formatted

In [ ]:
check = check_table_exists(dynamodb,table_name=TABLE_NAME)
check


Table VNA_history already exists


dynamodb.Table(name='VNA_history')

In [ ]:
# save_chat_history(
#     conversation_id="conv-123",
#     user_input="How can I learn Python?",
#     response="Python is a great language to learn. Start with tutorials and practice coding daily.",
#     start_time=datetime.datetime.now() - datetime.timedelta(minutes=5),
#     end_time=datetime.datetime.now(),
#     prompt_token=10,
#     completion_token=30,
#     total_token=40


{'conversation_id': 'conv-123', 'question_id': 'user-conv-123-1745228756'}

In [8]:
history = get_conversation_history("trungnnn")
print("Conversation history:", history)

Table VNA_history already exists
Retrieved 2 messages for conversation trungnnn
Conversation history: [{'id': 'b6c80be2-a382-4e7e-b8a4-93470c7e25a1', 'type': 'HUMAN-MESSAGE', 'question_id': 'user-trungnnn-1745250450', 'content': 'hanh ly ky gui bao can', 'created_at': '2025-04-21T15:47:30.443971+00:00', 'updated_at': '2025-04-21T15:47:41.689154+00:00', 'category_id': 'general', 'category_name': 'general', 'tokens': {'prompt': Decimal('2969'), 'completion': Decimal('406'), 'total': Decimal('3375')}, 'response_time': Decimal('11.245183'), 'source': None}, {'id': '451b3ca1-f60f-43e3-bc62-70262d6f58c7', 'type': 'AI-MESSAGE', 'question_id': 'user-trungnnn-1745250450', 'content': 'Hành lý ký gửi miễn cước trên các chuyến bay của Vietnam Airlines được quy định như sau:\n\n### **Kích thước và trọng lượng tối đa**\n- Tổng kích thước tối đa 3 chiều (dài x rộng x cao) của một kiện hành lý ký gửi không vượt quá **158cm**.\n- Trọng lượng tối đa tùy thuộc vào hạng vé và hành trình:\n\n### **Quy định

In [9]:
list_conversation("trungnnn")


Retrieved 2 messages for conversation trungnnn


[{'id': 'b6c80be2-a382-4e7e-b8a4-93470c7e25a1',
  'type': 'HUMAN-MESSAGE',
  'question_id': 'user-trungnnn-1745250450',
  'content': 'hanh ly ky gui bao can',
  'created_at': '2025-04-21T15:47:30.443971+00:00',
  'updated_at': '2025-04-21T15:47:41.689154+00:00',
  'category_id': 'general',
  'category_name': 'general',
  'tokens': {'prompt': Decimal('2969'),
   'completion': Decimal('406'),
   'total': Decimal('3375')},
  'response_time': Decimal('11.245183'),
  'source': None},
 {'id': '451b3ca1-f60f-43e3-bc62-70262d6f58c7',
  'type': 'AI-MESSAGE',
  'question_id': 'user-trungnnn-1745250450',
  'content': 'Hành lý ký gửi miễn cước trên các chuyến bay của Vietnam Airlines được quy định như sau:\n\n### **Kích thước và trọng lượng tối đa**\n- Tổng kích thước tối đa 3 chiều (dài x rộng x cao) của một kiện hành lý ký gửi không vượt quá **158cm**.\n- Trọng lượng tối đa tùy thuộc vào hạng vé và hành trình:\n\n### **Quy định theo hành trình**\n#### **Hành trình nội địa Việt Nam**\n- **Hạng Th

In [12]:
delete_conversation("conv-123")# 

Deleted item with ID: b5bacc12-4bca-4c62-8587-9f05c93b68d4
Deleted item with ID: f149c623-f17d-4cc5-b68a-7e0f3b4d309b
Deleted a total of 2 messages from conversation conv-123


2